# Case 2 - Clustering

### Import Modules

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage

### Load Data

In [2]:
data_path = '../../case_2/data/HR_data.csv'

data_pd = pd.read_csv(data_path)

### Preproccessing Data

In [ ]:
def preprocess_data(df, features_to_use):
    """
    Preprocess the data for clustering
    """
    # Select relevant features
    X = df[features_to_use].copy()
    
    # Handle missing values
    X = X.fillna(X.mean())
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, scaler

## Clustering Methods

### Hierarchical Clustering

In [ ]:
def apply_hierarchical_clustering(X, n_clusters=3):
    """
    Apply hierarchical clustering and visualize dendrogram
    """
    # Compute linkage matrix
    Z = linkage(X, method='ward')
    
    # Plot dendrogram
    plt.figure(figsize=(12, 6))
    dendrogram(Z)
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('Samples')
    plt.ylabel('Distance')
    plt.axhline(y=15, c='k', linestyle='--', label='Cut-off for {} clusters'.format(n_clusters))
    plt.legend()
    
    # Apply clustering with the chosen number of clusters
    clustering = AgglomerativeClustering(n_clusters=n_clusters)
    labels = clustering.fit_predict(X)
    
    # Calculate silhouette score
    sil_score = silhouette_score(X, labels)
    print(f"Silhouette Score for Hierarchical Clustering: {sil_score:.3f}")
    
    return labels

### K-means Clustering

In [ ]:
def apply_kmeans_clustering(X, max_clusters=10):
    """
    Apply K-means clustering and determine optimal k using elbow method
    """
    # Elbow method to find optimal k
    inertias = []
    silhouette_scores = []
    k_range = range(2, max_clusters + 1)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X)
        inertias.append(kmeans.inertia_)
        sil_score = silhouette_score(X, labels)
        silhouette_scores.append(sil_score)
    
    # Plot elbow method results
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(k_range, inertias, 'o-')
    plt.title('Elbow Method for K-means')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    
    plt.subplot(1, 2, 2)
    plt.plot(k_range, silhouette_scores, 'o-')
    plt.title('Silhouette Scores for K-means')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score')
    
    plt.tight_layout()
    
    # Choose optimal k based on silhouette scores
    optimal_k = k_range[silhouette_scores.index(max(silhouette_scores))]
    print(f"Optimal number of clusters based on silhouette score: {optimal_k}")
    
    # Apply K-means with optimal k
    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    
    return labels, kmeans.cluster_centers_, optimal_k

### GMM Clustering

In [ ]:
def apply_gmm_clustering(X, max_components=10):
    """
    Apply Gaussian Mixture Model clustering
    """
    # Find optimal number of components using BIC
    bic_scores = []
    n_components_range = range(1, max_components + 1)
    
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components, random_state=42)
        gmm.fit(X)
        bic_scores.append(gmm.bic(X))
    
    # Plot BIC scores
    plt.figure(figsize=(10, 6))
    plt.plot(n_components_range, bic_scores, 'o-')
    plt.title('BIC Scores for different numbers of GMM components')
    plt.xlabel('Number of components')
    plt.ylabel('BIC Score')
    
    # Choose optimal number of components based on BIC
    optimal_components = n_components_range[bic_scores.index(min(bic_scores))]
    print(f"Optimal number of GMM components based on BIC: {optimal_components}")
    
    # Apply GMM with optimal number of components
    gmm = GaussianMixture(n_components=optimal_components, random_state=42)
    labels = gmm.fit_predict(X)
    
    return labels, gmm.means_, optimal_components

### DBScan Clustering

In [ ]:
def apply_dbscan(X):
    """
    Apply DBSCAN clustering to identify outliers and natural clusters
    """
    # Estimate epsilon using nearest neighbors
    from sklearn.neighbors import NearestNeighbors
    
    nn = NearestNeighbors(n_neighbors=2)
    nbrs = nn.fit(X)
    distances, indices = nbrs.kneighbors(X)
    distances = np.sort(distances[:, 1])
    
    # Plot k-distance graph
    plt.figure(figsize=(10, 6))
    plt.plot(distances)
    plt.title('K-distance Graph')
    plt.xlabel('Points sorted by distance')
    plt.ylabel('Distance to 2nd nearest neighbor')
    
    # Choose epsilon from the elbow in the k-distance graph (for demonstration, we'll use a heuristic)
    epsilon = np.percentile(distances, 90) * 0.5
    
    # Apply DBSCAN
    dbscan = DBSCAN(eps=epsilon, min_samples=3)
    labels = dbscan.fit_predict(X)
    
    # Count number of clusters and noise points
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    
    print(f"Number of clusters: {n_clusters}")
    print(f"Number of noise points: {n_noise}")
    
    return labels


### Plotting functions

In [ ]:
def visualize_clusters_2d(X, labels, centers=None, method_name=""):
    """
    Visualize clusters in 2D using PCA or UMAP
    """
    # Apply dimensionality reduction
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    # Create a figure with two subplots
    plt.figure(figsize=(18, 8))
    
    # PCA plot
    plt.subplot(1, 2, 1)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis', alpha=0.7)
    if centers is not None:
        centers_pca = pca.transform(centers)
        plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c='red', s=100, marker='X')
    plt.title(f'PCA Visualization of {method_name} Clusters')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.colorbar(scatter, label='Cluster')
    
    plt.tight_layout()

In [ ]:
def analyze_clusters(df, labels, feature_cols):
    """
    Analyze and interpret the resulting clusters
    """
    # Add cluster labels to the original dataframe
    df['cluster'] = labels
    
    # Calculate cluster statistics
    cluster_stats = df.groupby('cluster')[feature_cols].mean()
    
    # Plot heatmap of cluster characteristics
    plt.figure(figsize=(15, 10))
    sns.heatmap(cluster_stats, cmap='coolwarm', center=0, annot=True, fmt='.2f')
    plt.title('Cluster Characteristics')
    plt.tight_layout()
    
    # Create radar plots for each cluster
    n_clusters = len(cluster_stats)
    n_features = len(feature_cols)
    
    # Normalize the features for radar plot
    scaler = StandardScaler()
    cluster_stats_scaled = pd.DataFrame(
        scaler.fit_transform(cluster_stats),
        index=cluster_stats.index,
        columns=cluster_stats.columns
    )
    
    # Create radar plot
    fig = plt.figure(figsize=(15, 10))
    theta = np.linspace(0, 2*np.pi, n_features, endpoint=False)
    theta = np.concatenate((theta, [theta[0]]))  # Close the loop
    
    for i in range(n_clusters):
        values = cluster_stats_scaled.iloc[i].values
        values = np.concatenate((values, [values[0]]))  # Close the loop
        
        ax = fig.add_subplot(2, (n_clusters+1)//2, i+1, polar=True)
        ax.plot(theta, values)
        ax.fill(theta, values, alpha=0.25)
        ax.set_xticks(theta[:-1])
        ax.set_xticklabels(feature_cols, size=8)
        ax.set_title(f'Cluster {i}')
    
    plt.tight_layout()
    
    return cluster_stats